In [ ]:
from qsharp_bridge import *
from qsharp_widgets import Circuit
import matplotlib.pyplot as plt
import os
import json
import requests
import qsharp
from dotenv import load_dotenv
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

load_dotenv()

In [ ]:
def visualize(counts):
    plt.figure(figsize=(8, 4))
    plt.bar(counts.keys(), counts.values())
    plt.tight_layout()
    plt.show()

In [ ]:
def run_on_quokka(shots, qsharp_expression) -> dict[str, int]:
    code_to_run = code + "\n\n" + f"""@EntryPoint()
operation Main() : (Result, Result) {{
    {qsharp_expression}
}}"""
    qasm_code = qasm2(code_to_run)

    req = {
        'script': qasm_code,
        'count': shots
    }

    quokka_url = f"{os.environ['QUOKKA_URL']}/qsim/qasm"
    result = requests.post(quokka_url, json=req, verify=False)
    data = json.loads(result.content)
    entries = [str(entry) for entry in data['result']['c']]
    counts = {entry: entries.count(entry) for entry in set(entries)}
    visualize(counts)
    return counts

In [ ]:
code = """
 open Microsoft.Quantum.Convert;
 open Microsoft.Quantum.Math;

    operation RunBell(op: ((Qubit, Qubit) => Unit)) : (Result, Result) {
            use (q1, q2) = (Qubit(), Qubit());
            PrepareBellState(q1, q2);
            op(q1, q2);
            H(q1);
            H(q2);
            let (r1, r2) = (M(q1), M(q2));
            return (r1, r2)
    }

        operation PrepareBellState(q1 : Qubit, q2: Qubit) : Unit {
        X(q1);
        X(q2);
        H(q1);
        CNOT(q1, q2);
    }

    // Bell
    operation Uab(q1 : Qubit, q2: Qubit) : Unit {
        R1(PI() / 3.0, q2);
    }

    operation Uac(q1 : Qubit, q2: Qubit)  : Unit {
        R1(2.0 * PI() / 3.0, q2);
    }

    operation Ubc(q1 : Qubit, q2: Qubit)  : Unit {
        R1(PI() / 3.0, q1);
        R1(2.0 * PI() / 3.0, q2);
    }
"""
qsharp.eval(code)

In [ ]:
Circuit(qsharp.circuit("RunBell(Uab)"))

In [ ]:
shots = 1024

In [ ]:
results_ab = run_on_quokka(shots, "RunBell(Uab)")
p_ab = results_ab["[0, 0]"] / shots
print(f"P(a+,b+) = {p_ab}")

In [ ]:
Circuit(qsharp.circuit("RunBell(Ubc)"))

In [ ]:
results_bc = run_on_quokka(shots, "RunBell(Ubc)")
p_bc = results_bc["[0, 0]"] / shots
print(f"P(b+,c+) = {p_bc}")

In [ ]:
Circuit(qsharp.circuit("RunBell(Uac)"))

In [ ]:
results_ac = run_on_quokka(shots, "RunBell(Uac)")
p_ac = results_ac["[0, 0]"] / shots
print(f"P(a+,c+) = {p_ac}")

In [ ]:
bell_result = p_ab + p_bc >= p_ac;
print(f"Bell's inequality satisfied? {bell_result}");